# VQE Calculation
File for the full ground state calculation of the VQE on the IBM Cloud

In [14]:
import numpy as np
from qiskit.algorithms.minimum_eigensolvers import VQE
from qiskit.algorithms.optimizers import NELDER_MEAD, GradientDescent
from qiskit_nature.circuit.library.ansatzes import UCC
from qiskit_nature.converters.second_quantization.qubit_converter import QubitConverter
from qiskit_nature.mappers.second_quantization import JordanWignerMapper
from qiskit.utils import algorithm_globals
from qiskit.primitives import Estimator
from qiskit.quantum_info import SparsePauliOp
from qiskit.circuit.library import TwoLocal
import datetime
import pylab

from qiskit import QuantumCircuit

from src.hamiltonian.FermionicHamiltonian import FermionicHamiltonian
from src.molecule.BeH2 import BeH2

## Creating the BeH2 molecule
The concrete molecule specifics can be found in the BeH2 class file

In [2]:
num_orbitals = 14
num_electrons = 6
molecule = BeH2(num_orbitals, num_electrons)

In [9]:
# create the hamiltonian object and get the operator
hamiltonian = FermionicHamiltonian(molecule)
operator = hamiltonian.get_hamiltonian()

TypeError: FermionicHamiltonian.__init__() missing 1 required positional argument: 'molecule'

In [10]:
# create estimator, optimizer, converter, ansatz
estimator = Estimator()
optimizer = NELDER_MEAD(maxiter=150, maxfev=150)
converter = QubitConverter(mapper=JordanWignerMapper(), two_qubit_reduction=True)
ucc_ansatz = UCC(qubit_converter=converter, num_spin_orbitals=14, num_particles=(0,6), excitations='sd', alpha_spin=True, beta_spin=True, max_spin_excitation=1, generalized=True, preserve_spin=True, reps=5)

In [11]:
print("\rOptimizer: {}        ".format(type(optimizer).__name__), end="")
algorithm_globals.random_seed = 50
# use ansatz TwoLocal
#ansatz2 = TwoLocal(rotation_blocks="ry", entanglement_blocks="cz")
counts = []
values = []
params = []

Optimizer: NELDER_MEAD        

In [12]:
# Define callback routine which will be invoked after every shot
def store_intermediate_result(eval_count, parameters, mean, std):
    time = datetime.datetime.now()
    print(f"Iteration done at {time}")
    counts.append(eval_count)
    values.append(mean)
    params.append(parameters)

## Actual Qiskit VQE Invocation

In [13]:
# Use VQE with estimator, ansatz, optimizer
vqe = VQE(estimator, ucc_ansatz, optimizer, callback=store_intermediate_result)
# invoke computation on the operator
time = datetime.datetime.now()
print(f"Starting calculation at: {time}")
result = vqe.compute_minimum_eigenvalue(operator=operator)
time = datetime.datetime.now()
print(f"Ended calculation at: {time}")


converge_counts = np.asarray(counts)
converge_vals = np.asarray(values)

print("\rOptimization complete");

Starting calculation at: 2023-04-26 10:02:26.462840


NameError: name 'op' is not defined

In [15]:
pylab.rcParams["figure.figsize"] = (12, 8)
pylab.plot(converge_counts, converge_vals, label=type(optimizer).__name__)
pylab.xlabel("Eval count")
pylab.ylabel("Energy")
pylab.title("Energy convergence for various optimizers")
pylab.legend(loc="upper right");

NameError: name 'converge_counts' is not defined

## Calculation of reference value
The eigenvalue calculation can also be done using a simple linear algebra solver in this case.

In [16]:
# Analytic solution
from qiskit.algorithms.minimum_eigensolvers import NumPyMinimumEigensolver
from qiskit.opflow import PauliSumOp

numpy_solver = NumPyMinimumEigensolver()
result = numpy_solver.compute_minimum_eigenvalue(operator=operator)
ref_value = result.eigenvalue.real
print(f"Reference value: {ref_value:.5f}")

NameError: name 'op' is not defined